<a href="https://colab.research.google.com/github/Z5-05/study_third_sem/blob/main/ml/Mephi_ML4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import re
import requests
import json
from tqdm import tqdm
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import warnings
warnings.filterwarnings("ignore")

In [95]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [96]:
!ls

drive  sample_data  seeds_dataset.txt


In [97]:
# path = "./drive/MyDrive/ML_mephi/seeds_dataset.txt"
path = "./seeds_dataset.txt"

with open("seeds_dataset.txt", "r") as f:
  text = f.read()

text = text.replace("\t\t", "\t")

array = []
for arr in tqdm(text.split("\n")):
  array.append([float(t) for t in arr.split('\t')])

100%|██████████| 210/210 [00:00<00:00, 185784.40it/s]


In [106]:
df.head()

,Площадь,Периметр,Компактность,Длина ядра,Ширина ядра,Коэффициент ассиметрии,Длина канавки ядра,Сорт
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1.0
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1.0
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1.0
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1.0
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1.0


In [99]:
df = pd.DataFrame(array)
df.columns = ["Площадь",
              "Периметр",
              "Компактность",
              "Длина ядра",
              "Ширина ядра",
              "Коэффициент ассиметрии",
              "Длина канавки ядра",
              "Сорт"]

In [100]:
raw_df = df.drop("Сорт", axis=1)

In [101]:
kmeans_model = KMeans(n_clusters=3, random_state=1).fit(raw_df)
labels = kmeans_model.labels_
base_silhouette = metrics.silhouette_score(raw_df, labels, metric='euclidean')
print(f"Базовое значение силуэта: {base_silhouette}")

Базовое значение силуэта: 0.4719337319126887


In [102]:
for cols in range(1, len(raw_df.columns)):
  pca = PCA(n_components=cols)
  df_pca = pd.DataFrame(pca.fit_transform(raw_df))
  kmeans_model = KMeans(n_clusters=3, random_state=1).fit(df_pca)
  labels = kmeans_model.labels_
  silhouette = metrics.silhouette_score(df_pca, labels, metric='euclidean')
  if silhouette > base_silhouette:
    print(f"Значение силуэта для PCA = {cols} лучше базового = {silhouette}")
    break
  print("Лучших вариантов не нашлось")


Значение силуэта для PCA = 1 лучше базового = 0.6426483839427913


In [103]:
for cols in range(1, len(raw_df.columns)):
  tsne = TSNE(n_components=cols)
  df_tsne = pd.DataFrame(pca.fit_transform(raw_df))
  kmeans_model = KMeans(n_clusters=3, random_state=1).fit(df_tsne)
  labels = kmeans_model.labels_
  silhouette = metrics.silhouette_score(df_tsne, labels, metric='euclidean')
  if silhouette > base_silhouette:
    print(f"Значение силуэта для PCA = {cols} лучше базового = {silhouette}")
    break
  print("Лучших вариантов не нашлось")


Значение силуэта для PCA = 1 лучше базового = 0.6426483839427913


In [104]:
kmeans = KMeans(init="k-means++", n_clusters=3).fit(raw_df)
labels = kmeans_model.labels_
silhouette = metrics.silhouette_score(raw_df, labels, metric='euclidean')
print(f"Значение силуэта типа инициализации k-means++ = {silhouette}")

kmeans = KMeans(init="random", n_clusters=3)
labels = kmeans_model.labels_
silhouette = metrics.silhouette_score(raw_df, labels, metric='euclidean')
print(f"Значение силуэта типа инициализации random = {silhouette}")

pca = PCA(n_components=3).fit(raw_df)
kmeans = KMeans(init=pca.components_, n_clusters=3)
labels = kmeans_model.labels_
silhouette = metrics.silhouette_score(raw_df, labels, metric='euclidean')
print(f"Значение силуэта типа инициализации PCA = {silhouette}")

Значение силуэта типа инициализации k-means++ = 0.454939565113377
Значение силуэта типа инициализации random = 0.454939565113377
Значение силуэта типа инициализации PCA = 0.454939565113377


In [105]:
for k in range(2, 11):
  kmeans_model = KMeans(n_clusters=k, random_state=1).fit(raw_df)
  labels = kmeans_model.labels_
  silhouette = metrics.silhouette_score(raw_df, labels, metric='euclidean')
  print(f"Значение силуэта для k = {k}: {silhouette}")

Значение силуэта для k = 2: 0.5182873410849098
Значение силуэта для k = 3: 0.4719337319126887
Значение силуэта для k = 4: 0.39646457495739773
Значение силуэта для k = 5: 0.36196414876416927
Значение силуэта для k = 6: 0.36866578093739255
Значение силуэта для k = 7: 0.35418936587734456
Значение силуэта для k = 8: 0.36246202436183916
Значение силуэта для k = 9: 0.36343055812500447
Значение силуэта для k = 10: 0.3399204354102194
